<a href="https://colab.research.google.com/github/karimamd/PoseEstimationForMobile/blob/master/Simple_resnet_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [0]:
from IPython.display import clear_output
!pip install pcloud
!pip install fs
!pip install yagmail
clear_output()

In [0]:
''' Forced restart of kernel
to check for installed packages
'''
import os
os._exit(00)


In [0]:
!ls

sample_data


**Rerun from here after restart**

In [0]:
import torch
from torchvision import models
from torch.utils.data.dataset import Dataset
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary

In [0]:
##################################
import os, time, sys, concurrent.futures
import datetime
from IPython.display import clear_output
import zipfile
import pandas as pd
import numpy as np
import math
import cv2
from tqdm import trange
from tqdm import tqdm
##################################
import yagmail
import urllib
import hashlib
from pcloud import PyCloud
from fs import opener
clear_output()

# Preprocessing

In [0]:
'''
A class encapsulating all functions used 
to utilize the cloud resources in order to 
upload and download saved models
'''
class cloud_:
  def __init__(self, username, password):
    
    self.username = username
    self.password = password
    self.pc = PyCloud(username, password)
    self.usr_enc = urllib.parse.quote(username, safe='')
    self.pass_enc = urllib.parse.quote(password, safe='')
    self.fs = opener.open_fs('pcloud://'+ self.usr_enc + ':' + self.pass_enc + '@/')

  def upload(self, dest, *files):
    self.pc.uploadfile(path= dest, files=[*files])
  
  def download(self, source, file):
    with open(file, 'wb') as f:
      self.fs.download(source, f)
  
  def exists(self, path):
    return self.fs.exists(path)
  
  def isCorrupt(self, base, ref):
    first_hash = self.fs.hash(ref, "md5")
    sec_hash = self.md5sum(base)
    return first_hash != sec_hash
      
  def md5sum(self, filename):
    md5 = hashlib.md5()
    with open(filename, 'rb') as f:
        for chunk in iter(lambda: f.read(512 * md5.block_size), b''):
            md5.update(chunk)
    return md5.hexdigest()
  
  def delete(self, f_path):
    self.fs.remove(f_path)
      


In [0]:
'''
A function to unzip the dataset(s) in short time using multiprocessing
'''
def fast_unzip(fn, dest):
  
    def unzip_member(zf, name, dest):
        while True:
            try:
                zf.extract(name, dest)
                break
            except OSError as e:
                if e.errno != os.errno.EEXIST:
                    raise
                time.sleep(2)
                pass

    with open(fn, 'rb') as f:
        zf = zipfile.ZipFile(f)
        futures = []
        with concurrent.futures.ThreadPoolExecutor() as executor:
            for member in zf.infolist():
                futures.append(executor.submit(unzip_member, zf,
                                               member.filename, dest))

            for future in concurrent.futures.as_completed(futures):
                future.result()

In [0]:
'''
A function that returns 19 feature maps containing heatmaps corresponding to 18 keypoints (nose, neck , arm ...etc)
and background of an image containing a single person
input:
annotations: an array of pairs of size 18, each pair is x,y coordinates of a keypoint
  
'''
def get_heatmap(annotations, desired_height, desired_width, scale=4):
    coco_parts = 18
    heatmap = np.zeros((coco_parts,
                        desired_height,
                        desired_width), dtype=np.float32)

    def put_heatmap(plane_idx, center, sigma):
        center_x, center_y = center
        _, height, width = heatmap.shape[:3]
        
        # vectorized_start
        
        th = 4.6052*(0.75)
        delta = math.sqrt(th * 2)

        x0 = int(max(0, center_x - delta * sigma + 0.5))
        y0 = int(max(0, center_y - delta * sigma + 0.5))

        x1 = int(min(width - 1, center_x + delta * sigma + 0.5))
        y1 = int(min(height - 1, center_y + delta * sigma + 0.5))

        exp_factor = 1 / 2.0 / sigma / sigma

        arr_heatmap = heatmap[plane_idx, y0:y1 + 1, x0:x1 + 1]
        y_vec = (np.arange(y0, y1 + 1) - center_y)**2
        x_vec = (np.arange(x0, x1 + 1) - center_x)**2
        xv, yv = np.meshgrid(x_vec, y_vec)
        arr_sum = exp_factor * (xv + yv)
        arr_exp = np.exp(-arr_sum)
        arr_exp[arr_sum > th] = 0
        heatmap[plane_idx, y0:y1 + 1, x0:x1 + 1] = 50 * np.maximum(arr_heatmap, arr_exp)

        # vectorized_end 

    for idx, point in enumerate(annotations):
        if point[0] < 0 or point[1] < 0:
            continue
        put_heatmap(idx, point, 6)

    
    #heatmap[-1] = np.clip(1 - np.amax(heatmap, axis=0), 0.0, 1.0)
    ch_ = np.array([], dtype= np.float32)
    
    if scale:
      for ch in heatmap:
        ch__ = cv2.resize(ch, (desired_height // scale, desired_width // scale), interpolation=cv2.INTER_AREA)
        ch_ = np.append(ch_,ch__)
      heatmap = np.reshape(ch_, (coco_parts, desired_height // scale, desired_width // scale ))
    
    return heatmap

In [0]:
!wget https://www.dropbox.com/s/ov5vb9mip2093ez/train_resized_.zip
!wget https://www.dropbox.com/s/nojosuy51vn34ft/train_col.csv
!wget https://www.dropbox.com/s/zw3aq8dmaalpnz5/train__.csv
##############################################################
!wget https://www.dropbox.com/s/h9wyzbqd58h63yk/val_col.csv
!wget https://www.dropbox.com/s/032nrgzm7fik5zf/val_resized.zip
##############################################################
fast_unzip('train_resized_.zip','/')
fast_unzip('val_resized.zip','/')
clear_output()

In [0]:
!ls

sample_data  train_col.csv  train_resized_.zip	val_col.csv
train	     train__.csv    val			val_resized.zip


# Model

In [0]:
model_path = '/models/overfitting/best_model/resnet_drop_fit_50.pth.tar'

In [0]:
"""
Model:
Pretrained model (e.g, resnet50) features except last 4 blocks
+
4 Convolutional Layers + Dropouts
+
3 Deconvolutional Layers + BatchNorm Layers
"""

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class TL_Model(nn.Module):
  
  def __init__(self):
    super(TL_Model, self).__init__()
    
    def features_extractor():
      resnet = models.resnet50(pretrained=True)
      ft_ext = torch.nn.Sequential(*(list(resnet.children())[:-2]))
      
      for param in ft_ext.parameters():
          param.requires_grad = True
      
      return ft_ext
    
    
    self.ft = features_extractor()
    self.cn0 = nn.Conv2d(2048, 1024, 1, 1, 0)
    self.cn1 = nn.Conv2d(1024, 512, 1, 1, 0)
    self.cn2 = nn.Conv2d(512, 256, 1, 1, 0)
    self.cn3 = nn.Conv2d(256, 112, 1, 1, 0)
    self.cn4 = nn.Conv2d(112, 56, 1, 1, 0)
    self.cn5 = nn.Conv2d(56, 18, 1, 1, 0)
    self.dc = nn.ConvTranspose2d(18,18,4,2,1)
    #self.dp =nn.Dropout2d(p=0.3, inplace=False)
    self.bn = nn.BatchNorm2d(18)

    
  def forward(self, x):
    x = F.relu(self.ft(x))
    for i in range(6):
      x = F.relu(self.__dict__['_modules']['cn'+str(i)](x))
      #x = self.dp(x)
    for i in range(5):
      x = F.relu(self.dc(x))
      x = self.bn(x)
        
      
    return x

model = TL_Model()
model.cuda()
summary(model,(3, 224, 224))
#print(model)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.torch/models/resnet50-19c8e357.pth
102502400it [00:00, 111925327.88it/s]


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [0]:
# !cp train_col.csv train_.csv

# df = pd.read_csv('train_.csv')

# ds = df.loc[(df == -1000).T.sum() <= 10]
# dh = ds.head(2000)
# dh.to_csv('train__.csv', index = False)

In [0]:
# from google.colab import files
# files.download('train__.csv')

In [0]:
class dataset_csv(Dataset):

    def __init__(self, csv_path, height, width, scale, transforms=None):
      """
      Args:
      csv_path (string): path to csv file
      height (int): image height
      width (int): image width
      transform: pytorch transforms for transforms and tensor conversion
      """
      self.data = pd.read_csv(csv_path)
      self.labels = np.asarray(self.data.iloc[:,1:])
      self.height = height
      self.width = width
      self.scale = scale
      self.transforms = transforms
      

    def __getitem__(self, index):

      image_label = self.labels[index]
      image_label = np.reshape(image_label,(-1,2))
      image_label = get_heatmap(image_label, self.height, self.width, self.scale)
      image = cv2.imread(self.data.iloc[index][0])
      image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
      if self.transforms is not None:
        img_as_tensor = self.transforms(image)
      return (img_as_tensor, image_label)

    def __len__(self):
        return len(self.data.index)
        



In [0]:
cl = cloud_('zyadabozaid@hotmail.com', 'kareemzyad')

In [0]:
def save_checkpoint(state, is_best, cl, filename, dest):
    torch.save(state, filename)
    cl.upload(dest, filename)
    if is_best:
        cl.upload(dest + '/best_model', filename)

In [0]:
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

In [0]:
img_params = (224, 224, 1)

channel_stats = dict(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
transformations = transforms.Compose([transforms.ToTensor(), transforms.Normalize(**channel_stats)])

train_dataset_from_csv = dataset_csv('train_col.csv', *img_params, transformations)
val_dataset_from_csv = dataset_csv('val_col.csv', *img_params, transformations)

train_dataset_loader = \
    torch.utils.data.DataLoader(dataset=train_dataset_from_csv,
                                batch_size=64, shuffle=True)
val_dataset_loader = \
    torch.utils.data.DataLoader(dataset=val_dataset_from_csv,
                                batch_size=64, shuffle=True)

# choose Adam as an optimizer with learning rate

learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=150, gamma=0.1)
criterion = nn.MSELoss()

num_epochs = 350

start_epoch = 0

best_train_loss = sys.float_info.max

# stats = {}

model_file = os.path.split(os.path.abspath(model_path))[-1]
model_dir = os.path.split(os.path.abspath(model_path))[0]

resume = True

st_loss= 0.0
end_loss= 0.0

if resume:
    if cl.exists(model_path):
        cl.download(model_path, model_file)
        print("=> loading checkpoint '{}'".format(model_file))
        checkpoint = torch.load(model_file)
        start_epoch = checkpoint['epoch']
        best_train_loss = checkpoint['best_train_loss']
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        print("=> loaded checkpoint '{}' (epoch {})".format(model_file,
                checkpoint['epoch']))
    else:
        print('No saved model found on cloud')

for epoch in range(start_epoch, num_epochs):

    start_time = time.time()

    running_loss = 0.0
    eval_loss = 0.0
    scheduler.step()
    
    model.train()
    
    with trange(len(train_dataset_loader)) as t:
      for (images, heatmaps_true) in train_dataset_loader:

          images = images.to(device)
          heatmaps_true = heatmaps_true.to(device)

          optimizer.zero_grad()

          heatmaps_pred = model(images)
          #print(heatmaps_pred.size())
          ''''
          
          ''''
          loss = criterion(heatmaps_pred, heatmaps_true)

          # backprop. + optimize

          loss.backward()

          optimizer.step()

          # compute loss

          running_loss += loss.item() * images.size(0)
          
          metrics = {'train_loss':loss.item(), 'lr':get_lr(optimizer)}
          
          t.set_description('Epoch {}/{}'.format(epoch, num_epochs - 1))
          t.set_postfix(metrics)
          t.update()
    
    model.eval()
    
    with trange(len(val_dataset_loader)) as t:
      for (images, heatmaps_true) in val_dataset_loader:

          images = images.to(device)
          heatmaps_true = heatmaps_true.to(device)

          with torch.no_grad():

              heatmaps_pred = model(images)
              loss = criterion(heatmaps_pred, heatmaps_true)

              eval_loss += loss.item() * images.size(0)
          
          metrics = {'val_loss':loss.item(), 'lr':get_lr(optimizer)}
          
          t.set_description('Epoch {}/{}'.format(epoch, num_epochs - 1))
          t.set_postfix(metrics)
          t.update()
    
    val_loss = eval_loss / len(val_dataset_loader.dataset)
    epoch_loss = running_loss / len(train_dataset_loader.dataset)

    is_best = epoch_loss < best_train_loss
    best_train_loss = min(epoch_loss, best_train_loss)
    
    if epoch == start_epoch:
      st_loss = epoch_loss
    if epoch == num_epochs-1:
      end_loss = epoch_loss
      
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_train_loss': best_train_loss,
        'optimizer': optimizer.state_dict(),
        }, is_best, cl, model_file, model_dir)
    
#     stats['Epoch '+ str(epoch)] = (epoch_loss, val_loss, (time.time() - start_time) / 60)


# 
yag = yagmail.SMTP('pose.project.models@gmail.com', 'poseproject19')
# contents = ["Hello Kareem \n","Finished training of Overfitting ResNet Model Deconv. @ ", str(datetime.datetime.now()), " LTZ."
#            ,"\n=> Start Training Loss: ",st_loss," End Training Loss: ",end_loss]
# yag.send('karimamd95@gmail.com', 'Overfitting ResNet Model Training Finish', contents)



In [0]:
# os.environ["CUDA_VISIBLE_DEVICES"] = '3'